In [1]:
!pip install google-generativeai

In [2]:
from random import randint
from typing import Iterable

import google.generativeai as genai
from google.api_core import retry

from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [3]:
order = []  # The in-progress order.
placed_order = []  # The confirmed, completed order.

def add_to_order(drink: str, modifiers: Iterable[str] = ()) -> None:
  """Adds the specified drink to the customer's order, including any modifiers."""
  order.append((drink, modifiers))


def get_order() -> Iterable[tuple[str, Iterable[str]]]:
  """Returns the customer's order."""
  return order


def remove_item(n: int) -> str:
  """Remove the nth (one-based) item from the order.

  Returns:
    The item that was removed.
  """
  item, modifiers = order.pop(int(n) - 1)
  return item


def clear_order() -> None:
  """Removes all items from the customer's order."""
  order.clear()


def confirm_order() -> str:
  """Asks the customer if the order is correct.

  Returns:
    The user's free-text response.
  """

  print('Your order:')
  if not order:
    print('  (no items)')

  for drink, modifiers in order:
    print(f'  {drink}')
    if modifiers:
      print(f'   - {", ".join(modifiers)}')

  return input('Is this correct? ')


def place_order() -> int:
  """Submit the order to the kitchen.

  Returns:
    The estimated number of minutes until the order is ready.
  """
  placed_order[:] = order.copy()
  clear_order()

  # TODO(you!): Implement coffee fulfilment.
  return randint(1, 10)

In [4]:
user_info = {}  # Dictionary to store user information
interests = []  # List to store user's interests

def add_name(name: str) -> None:
    """Adds the user name to the dictionary"""
    user_info['name'] = name

def add_field(status: str, field: str) -> None:
    """Adds the user status studying/working in the status and in which field they are studying/working  ."""
    user_info['status'] = status
    user_info['field'] = field

def get_interests(new_interest: str) -> None:
    """Adds and stores the user interests."""
    interests.append(new_interest)

def confirm_details() -> dict:
    """Confirms User details and closes chats"""
    """Returns user details"""
    print(user_info)

In [5]:
FEATUREHIVE_BOT_PROMPT = """
You are Nexus a welcome and support system for FeatureHive, a unified platform for workshops, speaker sessions, collaboration networks, and more. You are restricted to providing information and assistance related to the platform's features and services. Do not discuss anything outside of these topics.

Your goal is to greet users get their name and store it in database using add_name ,then Asks the user if they are studying or working and in which field and stores the information using add_field
then ask the user about their interests like self improvement technical finance non tech etc add the interests in the list using get_interests and at the end just tell users to proceed to platform to explore more.
once you get all the details as per flow call the confirm_details to confirm user registration.
Key tasks include:
1. Greeting the user and asking for their name.
2. Asking if the user is currently studying or working, and in which field.
3. Inquiring about their interests related to workshops, speaker sessions, and collaboration networks.
4. Providing tailored information based on their interests.
5. Assisting users in joining and navigating collaboration networks.
6. Providing technical support for any platform-related issues.
7. Answering any questions about the platform's features and how to use them effectively.

Example Flow:(keep it unique every time)
1. "Welcome to FeatureHive! What's your name?"
2. "Nice to meet you, [Name]! Are you currently studying or working? In which field?"
3. "Great! What are your interests related to workshops, speaker sessions, and collaboration networks?"
4. "Thank you, [Name]! Based on your interest in [Interests], our platoform has workshops and seminars and professional guidance which you can explore further as you move on.
then make the users proceed

Remember, your focus is on providing a seamless and helpful experience for the users as they navigate FeatureHive.
"""


In [14]:
ordering_system = [add_name, add_field, get_interests, confirm_details]

# Toggle this to switch between Gemini 1.5 with a system instruction, or Gemini 1.0 Pro.
use_sys_inst = False

model_name = 'gemini-1.5-flash-latest' if use_sys_inst else 'gemini-1.0-flash-latest'

if use_sys_inst:
  model = genai.GenerativeModel(
      model_name, tools=ordering_system, system_instruction=FEATUREHIVE_BOT_PROMPT)
  convo = model.start_chat(enable_automatic_function_calling=True)

else:
  model = genai.GenerativeModel('gemini-1.5-flash-latest', tools=ordering_system)
  convo = model.start_chat(
      history=[
          {'role': 'user', 'parts': [FEATUREHIVE_BOT_PROMPT]},
          {'role': 'model', 'parts': ['OK I understand. I will do my best!']}
        ],
      enable_automatic_function_calling=True)


@retry.Retry(initial=30)
def send_message(message):
  return convo.send_message(message)


placed_order = []
order = []

In [15]:
from IPython.display import display, Markdown

print('Welcome to Nexus bot!\n\n')

while not user_info:
  response = send_message(input('> '))
  display(Markdown(response.text))


print('\n\n')
print('[barista bot session over]')
print()
print('Your order:')
print(f'  {placed_order}\n')
print('- Thanks for using Barista Bot!')

Welcome to Nexus bot!


> hi


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Request contains an invalid argument.